In [1]:
import os, sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

from utils.utils import Create_Similarity_Model

2022-07-06 13:53:48.428500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 13:53:48.428515: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Now we will create our model

In [2]:
n_nonsimilar = 1
n_similar = 1

### We will look for similarity in the last n_similar parameters of the inputs.

model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 similarity_layer_1 (Dense)     (None, 1)            1           ['lambda[0][0]']                 
                                                                                              

2022-07-06 13:53:51.163550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 13:53:51.163933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 13:53:51.163973: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-06 13:53:51.164009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-06 13:53:51.164044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## Define the loss, optimizer and metrics

In [3]:
loss = 'mean_squared_error'
optimizer = 'adam'
metrics = ['mean_squared_error']

model.compile(loss = loss, optimizer = optimizer, metrics = metrics)

## Fabricate our data simulating the laminar flow

In [4]:
possible_re_tau = np.linspace(start = 1, stop = 100, num = 100)
Re_tau = []
Y_ =[]
U_ = []
dic_x = {}
dic_y = {}

for re_tau in possible_re_tau:
    possible_y_= np.linspace(start = 0.1, stop = re_tau, num = 100)
    for y_ in possible_y_:
        Y_.append(y_)
        Re_tau.append(re_tau)
        u_ = y_ - ((1 / (2*re_tau)) * ((y_) ** 2))
        U_.append(u_)

Y_ = np.log(Y_)
Re_tau = np.log(Re_tau)

dic_x['y+'] = Y_
dic_x['re_tau'] = Re_tau
dic_y['u+'] = U_

nonsimilar_keys = ['y+']

xtrain = pd.DataFrame.from_dict(dic_x).values
ytrain = pd.DataFrame.from_dict(dic_y).values

In [5]:
model.fit(xtrain, ytrain, epochs = 1000, verbose = 1)

Epoch 1/1000
313/313 [==============================] - 1s 878us/step - loss: 358.6801 - mean_squared_error: 358.6801
Epoch 2/1000
313/313 [==============================] - 0s 903us/step - loss: 190.8252 - mean_squared_error: 190.8252
Epoch 3/1000
313/313 [==============================] - 0s 903us/step - loss: 144.7307 - mean_squared_error: 144.7307
Epoch 4/1000
313/313 [==============================] - 0s 866us/step - loss: 126.5067 - mean_squared_error: 126.5067
Epoch 5/1000
313/313 [==============================] - 0s 867us/step - loss: 116.0855 - mean_squared_error: 116.0855
Epoch 6/1000
313/313 [==============================] - 0s 871us/step - loss: 108.0653 - mean_squared_error: 108.0653
Epoch 7/1000
313/313 [==============================] - 0s 864us/step - loss: 101.7079 - mean_squared_error: 101.7079
Epoch 8/1000
313/313 [==============================] - 0s 870us/step - loss: 71.9714 - mean_squared_error: 71.9714
Epoch 9/1000
313/313 [==============================] - 0s

KeyboardInterrupt: 

In [11]:
for i in range(n_nonsimilar):
    similarity_exponents = model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
    print('Exponents multiplicating inside the function for ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

multiplication_exponents = model.get_layer('multiplication_layer').weights[0][0].numpy()
print('Exponents multiplicating outside the function: ' + str(multiplication_exponents))

Exponents multiplicating inside the function for y+: [-0.9994309]
Exponents multiplicating outside the function: [0.99976444]
